In [1]:
import duckdb
import pandas as pd
import csv
import re

In [2]:
# import /workspaces/dec-etl-project/data/BASE_OUTPUT_DIRECTORY/etl_stats.txt_7 using the csv library
with open('/workspaces/dec-etl-project/data/BASE_OUTPUT_DIRECTORY/etl_stats_7.txt', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    data = list(reader)

# convert to pandas dataframe
df = pd.DataFrame(data)

df.columns = ['string', 'timestamp', 'message']

# for each row, create a column that only has characters between "[" and "]" and name it "timestamp". convert "timestamp" to a datetime object
df['timestamp'] = df['string'].str.extract(r'\[(.*?)\]', expand=False)
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [3]:
df

,string,timestamp,message
0,[2023-02-10 17:19:39]CMS_ETL starting,2023-02-10 17:19:39,None
1,[2023-02-10 17:19:39]BASE_SYNPUF_INPUT_DIRECTO...,2023-02-10 17:19:39,None
2,[2023-02-10 17:19:39]BASE_OUTPUT_DIRECTORY ...,2023-02-10 17:19:39,None
3,[2023-02-10 17:19:39]BASE_ETL_CONTROL_DIRECTOR...,2023-02-10 17:19:39,None
4,[2023-02-10 17:19:39]--Table_ID_Values loaded:,2023-02-10 17:19:39,None
...,...,...,...
70,[2023-02-10 17:40:50],2023-02-10 17:40:50,None
71,[2023-02-10 17:40:50],2023-02-10 17:40:50,None
72,[2023-02-10 17:40:50],2023-02-10 17:40:50,None
73,[2023-02-10 17:40:50],2023-02-10 17:40:50,None


In [4]:
# create a colummn that only has the characters after the first "]"
df['message'] = df['string'].str.extract(r'\](.*)', expand=False)

# move all the rows that ave not null in df[1] to the message column
df['message'] = df['message'].mask(df['string'].notnull(), df['string'])

# delete all columns not named timestamp or message
df = df[['timestamp', 'message']]

In [5]:
df

,timestamp,message
0,2023-02-10 17:19:39,[2023-02-10 17:19:39]CMS_ETL starting
1,2023-02-10 17:19:39,[2023-02-10 17:19:39]BASE_SYNPUF_INPUT_DIRECTO...
2,2023-02-10 17:19:39,[2023-02-10 17:19:39]BASE_OUTPUT_DIRECTORY ...
3,2023-02-10 17:19:39,[2023-02-10 17:19:39]BASE_ETL_CONTROL_DIRECTOR...
4,2023-02-10 17:19:39,[2023-02-10 17:19:39]--Table_ID_Values loaded:
...,...,...
70,2023-02-10 17:40:50,[2023-02-10 17:40:50]
71,2023-02-10 17:40:50,[2023-02-10 17:40:50]
72,2023-02-10 17:40:50,[2023-02-10 17:40:50]
73,2023-02-10 17:40:50,[2023-02-10 17:40:50]


In [6]:
# Read the text file
with open('/workspaces/dec-etl-project/data/BASE_OUTPUT_DIRECTORY/etl_stats_7.txt', 'r') as f:
    lines = f.readlines()

# Create empty lists to store the extracted timestamps and messages
timestamps = []
messages = []

# Use a regular expression pattern to extract the timestamp and message from each line
pattern = re.compile(r'\[(.*?)\](.*)')
for line in lines:
    match = pattern.search(line)
    if match:
        timestamps.append(match.group(1))
        messages.append(match.group(2))

# Create a dataframe from the extracted data
df = pd.DataFrame({'timestamp': timestamps, 'message': messages})

df

,timestamp,message
0,2023-02-10 17:19:39,CMS_ETL starting
1,2023-02-10 17:19:39,BASE_SYNPUF_INPUT_DIRECTORY =/workspaces/d...
2,2023-02-10 17:19:39,BASE_OUTPUT_DIRECTORY =/workspaces/d...
3,2023-02-10 17:19:39,BASE_ETL_CONTROL_DIRECTORY =/workspaces/d...
4,2023-02-10 17:19:39,--Table_ID_Values loaded:
...,...,...
70,2023-02-10 17:40:50,\tFile: payer_plan_period ...
71,2023-02-10 17:40:50,\tFile: person ...
72,2023-02-10 17:40:50,\tFile: procedure_cost ...
73,2023-02-10 17:40:50,\tFile: procedure_occurrence ...


In [18]:
import re
import pandas as pd
import os

def extract_etl_stats(base_output_directory):
    pattern = re.compile(r'\[(.*?)\](.*)')
    etl_stats_df = pd.DataFrame(columns=['Timestamp', 'Message'])
    
    for i in range(1, 21):
        log_file_path = os.path.join(base_output_directory, f'etl_stats_{i}.txt')
        if os.path.exists(log_file_path):
            with open(log_file_path, 'r') as log_file:
                log_contents = log_file.read()
                matches = pattern.finditer(log_contents)
                for match in matches:
                    etl_stats_df = etl_stats_df.append({'Timestamp': match.group(1), 'Message': match.group(2)}, ignore_index=True)
    return etl_stats_df

# use con.execute to add etl_stats_df to the metadata table
con.execute('create table if not exists synpuf.metadata (timestamp timestamp, message varchar)')
con.execute('insert into synpuf.metadata select * from table(etl_stats_df)')


/tmp/ipykernel_68708/450019974.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  etl_stats_df = etl_stats_df.append({'Timestamp': match.group(1), 'Message': match.group(2)}, ignore_index=True)
/tmp/ipykernel_68708/450019974.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  etl_stats_df = etl_stats_df.append({'Timestamp': match.group(1), 'Message': match.group(2)}, ignore_index=True)
/tmp/ipykernel_68708/450019974.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  etl_stats_df = etl_stats_df.append({'Timestamp': match.group(1), 'Message': match.group(2)}, ignore_index=True)
/tmp/ipykernel_68708/450019974.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. U

,Timestamp,Message
0,2023-02-10 17:19:39,CMS_ETL starting
1,2023-02-10 17:19:39,BASE_SYNPUF_INPUT_DIRECTORY =/workspaces/d...
2,2023-02-10 17:19:39,BASE_OUTPUT_DIRECTORY =/workspaces/d...
3,2023-02-10 17:19:39,BASE_ETL_CONTROL_DIRECTORY =/workspaces/d...
4,2023-02-10 17:19:39,--Table_ID_Values loaded:
...,...,...
148,2023-02-10 21:50:45,\tFile: person ...
149,2023-02-10 21:50:45,\tFile: procedure_cost ...
150,2023-02-10 21:50:45,\tFile: procedure_occurrence ...
151,2023-02-10 21:50:45,\tFile: provider ...


In [17]:
# call the function
extract_etl_stats('/workspaces/dec-etl-project/data/BASE_OUTPUT_DIRECTORY')

/tmp/ipykernel_68708/589516557.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  etl_stats_df = etl_stats_df.append({'Timestamp': match.group(1), 'Message': match.group(2)}, ignore_index=True)
/tmp/ipykernel_68708/589516557.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  etl_stats_df = etl_stats_df.append({'Timestamp': match.group(1), 'Message': match.group(2)}, ignore_index=True)
/tmp/ipykernel_68708/589516557.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  etl_stats_df = etl_stats_df.append({'Timestamp': match.group(1), 'Message': match.group(2)}, ignore_index=True)
/tmp/ipykernel_68708/589516557.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. U

,Timestamp,Message
0,2023-02-10 17:19:39,CMS_ETL starting
1,2023-02-10 17:19:39,BASE_SYNPUF_INPUT_DIRECTORY =/workspaces/d...
2,2023-02-10 17:19:39,BASE_OUTPUT_DIRECTORY =/workspaces/d...
3,2023-02-10 17:19:39,BASE_ETL_CONTROL_DIRECTORY =/workspaces/d...
4,2023-02-10 17:19:39,--Table_ID_Values loaded:
...,...,...
148,2023-02-10 21:50:45,\tFile: person ...
149,2023-02-10 21:50:45,\tFile: procedure_cost ...
150,2023-02-10 21:50:45,\tFile: procedure_occurrence ...
151,2023-02-10 21:50:45,\tFile: provider ...


In [ ]:
etl_stats_df